# Web Scraping: Website Scrap

## Iswarya SIVASUBRAMANIAM DIA1

In [75]:
import bs4
from urllib import request
import pandas as pd

import requests
import io
import pdfplumber
import re

In [16]:
#URL du site à scraper
url_ligue_1="https://www.fiches-auto.fr/articles-auto/voiture-electrique/s-852-comparatif-des-voitures-electriques.php"

In [17]:
request_text = request.urlopen(url_ligue_1).read()

In [18]:
htmlpage = bs4.BeautifulSoup(request_text, "html")

In [19]:
# création d'une  liste pour stocker les élements du site
td_elements=[]

In [20]:
tables = htmlpage.find_all('table')

In [21]:
# Nous scrapons les éléments de chaque cellules des tables du site internet
#Les 3 premières tables de notre site ne continennet pas les informations interessantes:
for index, table in enumerate(tables[:-3]):

    print(f"Table {index + 1}:") 
    rows = table.find_all('tr')
    is_first_row=True
    for row in rows:
        if is_first_row:
            is_first_row = False
            continue 
        columns = row.find_all(['th', 'td'])
        for col in columns:
            td_elements.append(col.text.strip())
            print(col.text.strip(), end='\t')
        print() 
    print()  

Table 1:
Ami (8 ch)	0Sec.	45km/h	0Dm3	5.5kWh	70km	

Table 2:
C-Zero (67 ch)	15.9Sec.	130km/h	166Dm3	14.5kWh	100km	
Spring (45 ch)	19.1Sec.	125km/h	270Dm3	26.8kWh	230km	
Spring (65 ch)	13.7Sec.	125km/h	270Dm3	26.8kWh	220km	
500e (95 ch)	9.5Sec.	135km/h	185Dm3	21.3kWh	190km	
500e (118 ch)	9Sec.	150km/h	185Dm3	37.3kWh	321km	
T03 (109 ch)	12Sec.	130km/h	210Dm3	41.3kWh	417km	
Leaf (109 ch)	11.5Sec.	144km/h	370Dm3	22kWh	199km (NEDC)	
Ion (67 ch)	15.9Sec.	130km/h	166Dm3	14.5kWh	100km	
	0 à 100	V max	Coffre	Bat.kWh	Autonomie	
Twingo III (82 ch)	12.9Sec.	135km/h	240Dm3	22kWh	190km	
Zoe (88 ch)	13.5Sec.	135km/h	338Dm3	22kWh	210km (NEDC)	
Zoe (77 ch)	15.5Sec.	135km/h	338Dm3	41kWh	400km (NEDC)	
Zoe (88 ch)	13.2Sec.	135km/h	338Dm3	41kWh	300km	
Zoe (92 ch)	13.2Sec.	135km/h	338Dm3	41kWh	317km	
Zoe (108 ch)	11.4Sec.	135km/h	338Dm3	41kWh	300km	
Mii EV (83 ch)	12.3Sec.	130km/h	251Dm3	32.3kWh	258km	
Fortwo (82 ch)	11.5Sec.	130km/h	260Dm3	17.6kWh	135km	
E-Up! (82 ch)	12.4Sec.	130km/h	250Dm3	18.7kWh	160km 

In [22]:
# cération d'un DataFrame pour stocker les élements du site
Modèle_voiture = []
Acceleration = []
Vmax = []
Coffre = []
Batterie = []
Autonomie = []
for i in range(0, len(td_elements), 6):  
    car_name = td_elements[i]
    Acceleration_value = td_elements[i + 1]
    Vmax_value = td_elements[i + 2]
    Coffre_value = td_elements[i + 3]
    Batterie_value = td_elements[i + 4]
    Autonomie_value = td_elements[i + 5]

    Modèle_voiture.append(car_name)
    Acceleration.append(Acceleration_value)
    Vmax.append(Vmax_value)
    Coffre.append(Coffre_value)
    Batterie.append(Batterie_value)
    Autonomie.append(Autonomie_value)

data = {
    'Car Name': Modèle_voiture,
    'Acceleration 0 à 100(Sec)': Acceleration,
    'Max Speed (km/h)': Vmax,
    'Cargo Volume (Dm3)': Coffre,
    'Battery (kWh)': Batterie,
    'Autonomie (km)': Autonomie,
}
df = pd.DataFrame(data)

In [23]:
df

,Car Name,Acceleration 0 à 100(Sec),Max Speed (km/h),Cargo Volume (Dm3),Battery (kWh),Autonomie (km)
0,Ami (8 ch),0Sec.,45km/h,0Dm3,5.5kWh,70km
1,C-Zero (67 ch),15.9Sec.,130km/h,166Dm3,14.5kWh,100km
2,Spring (45 ch),19.1Sec.,125km/h,270Dm3,26.8kWh,230km
3,Spring (65 ch),13.7Sec.,125km/h,270Dm3,26.8kWh,220km
4,500e (95 ch),9.5Sec.,135km/h,185Dm3,21.3kWh,190km
...,...,...,...,...,...,...
218,EQS (523 ch),4.3Sec.,210km/h,610Dm3,107.8kWh,651km
219,Taycan (625 ch),3.2Sec.,260km/h,366Dm3,83.7kWh,450km
220,Taycan (625 ch),2.8Sec.,260km/h,366Dm3,83.7kWh,412km
221,Taycan Cross Turismo (625 ch),3.3Sec.,250km/h,405Dm3,83.7kWh,452km


## Scraping from a PDF

In [96]:
desired_content=""
pdf_url = "http://johnhwman.free.fr/Divers/Comparatif_VE.pdf"
response = requests.get(pdf_url)
desired_portion=""
third=""

with pdfplumber.open(io.BytesIO(response.content)) as pdf:
    for page_number in range(len(pdf.pages)):
        page = pdf.pages[page_number]
        text = page.extract_text()

        if page_number==0:
            start_pattern1 = r"Citroën Ami"
            end_pattern1=r"37 550 - 47 150 €"
            start_index1 = re.search(start_pattern1, text)
            end_index1 = re.search(end_pattern1, text)
            start_pattern2 = r"Tesla Model 3"
            end_pattern2=r"42 990 €"
            start_index2 = re.search(start_pattern2, text)
            end_index2 = re.search(end_pattern2, text)
            desired_portion = text[start_index1.start():end_index1.end()] 
            print(f"Page {page_number + 1}:\n{desired_portion}\n")
            desired_portion +=text[start_index2.start():end_index2.end()]
            print(f"Page {page_number + 1}:\n{desired_portion}\n")

        if page_number==1:
            start_pattern1 = r"Mazda MX-30"
            end_pattern1=r"57 500 €"
            start_index1 = re.search(start_pattern1, text)
            end_index1 = re.search(end_pattern1, text)
            start_pattern2 = r"Ford Mustang"
            end_pattern2=r"185 750 €"
            start_index2 = re.search(start_pattern2, text)
            end_index2 = re.search(end_pattern2, text)
            desired_portion += text[start_index1.start():end_index1.end()] 
            desired_portion +=text[start_index2.start():end_index2.end()]
            start_pattern3 = r"Byton M-Byte"
            end_pattern3=r"43000 € ?"
            start_index3 = re.search(start_pattern3, text)
            end_index3 = re.search(end_pattern3, text)
            desired_portion +=text[start_index3.start():end_index3.end()]
        if page_number == 2:
            start_pattern1 = r"Thunder Power Sedan"
            end_pattern1=r"78000 € ?"
            start_index1 = re.search(start_pattern1, text)
            end_index1 = re.search(end_pattern1, text)
            desired_portion +=text[start_index1.start():end_index1.end()]

Page 1:
Citroën Ami (2020) (FWD) >14 ans (2020) 5,8kWh / 5,35kWh 12 ch / 9 kW / 40 Nm N/D 45 km/h 471 kg / ? / ? 70-75 km / 78,6 Wh/km N/D N/D 1,84kW AC (Prise domestique) 0,82€ @ 1,8kW 1,09 € 6 000 €
Lightyear One (2022) (AWD) 60kWh / 60kWh 136 ch / 100 kW / 1200 Nm 10 sec. 150 km/h 1300 kg /0,2/0,46 725 km / 83 Wh/km 875-505 km / 69-119 Wh/km 565-405 km / 106-148 Wh/km Type 2 - 22kW AC / CCS 75 kW DC 8,85€ @ 22kW 1,18 € 149 000 €
Renault Twingo Electric (RWD) (2020) 23 kWh / 21,3 kWh 82 ch / 60 kW / 160 Nm 12,6 sec. 135 km/h 1208kg/0,33/ 0,72 190 km / 112,1 Wh/km 205-135 km / 104-158 Wh/km 120-90 km / 178-237 Wh/km Type 2 Accélérée - 22kW AC 3,14€ @ 22kW 1,65 € 15 350 - 20 450 €
Tesla Model 3 - Std range+ (NCA) (RWD)(2019) 54kWh / 51kWh 325 ch / 239 kW / 420 Nm 5,6 sec. 225 km/h 1700kg/0,23/0,52 448 km / 113,8 Wh/km 515-330 km / 99-155 Wh/km 330-250 km / 155-204 Wh/km Type 2 - 11kW AC / CCS 170kW DC 7,52€@11kW / 12,24€ (SUC) 1,67€/2,73€ (SUC) 43 800 €
Fiat 500e Hatchback 42 kWh (FWD)

In [97]:
desired_portion

'Citroën Ami (2020) (FWD) >14 ans (2020) 5,8kWh / 5,35kWh 12 ch / 9 kW / 40 Nm N/D 45 km/h 471 kg / ? / ? 70-75 km / 78,6 Wh/km N/D N/D 1,84kW AC (Prise domestique) 0,82€ @ 1,8kW 1,09 € 6 000 €\nLightyear One (2022) (AWD) 60kWh / 60kWh 136 ch / 100 kW / 1200 Nm 10 sec. 150 km/h 1300 kg /0,2/0,46 725 km / 83 Wh/km 875-505 km / 69-119 Wh/km 565-405 km / 106-148 Wh/km Type 2 - 22kW AC / CCS 75 kW DC 8,85€ @ 22kW 1,18 € 149 000 €\nRenault Twingo Electric (RWD) (2020) 23 kWh / 21,3 kWh 82 ch / 60 kW / 160 Nm 12,6 sec. 135 km/h 1208kg/0,33/ 0,72 190 km / 112,1 Wh/km 205-135 km / 104-158 Wh/km 120-90 km / 178-237 Wh/km Type 2 Accélérée - 22kW AC 3,14€ @ 22kW 1,65 € 15 350 - 20 450 €\nTesla Model 3 - Std range+ (NCA) (RWD)(2019) 54kWh / 51kWh 325 ch / 239 kW / 420 Nm 5,6 sec. 225 km/h 1700kg/0,23/0,52 448 km / 113,8 Wh/km 515-330 km / 99-155 Wh/km 330-250 km / 155-204 Wh/km Type 2 - 11kW AC / CCS 170kW DC 7,52€@11kW / 12,24€ (SUC) 1,67€/2,73€ (SUC) 43 800 €\nFiat 500e Hatchback 42 kWh (FWD) (2

# Dernier site

In [28]:
import requests
import bs4
import pandas as pd

url = "https://www.numerama.com/vroom/599805-voitures-electriques-tous-les-modeles-disponibles-en-france-en-2020.html"

response = requests.get(url)
html_content = response.text
soup = bs4.BeautifulSoup(html_content, 'html.parser')

# Find all the <h2> elements with class 'wp-block-heading'
modele_elements = soup.find_all('h2', class_='wp-block-heading')


modeles = []
puissances = []
autonomies = []
prix = []
bonus = []

for modele_element in modele_elements:
    modele = modele_element.text.strip()
    modeles.append(modele)
    print(modele)

    figure_element = modele_element.find_next('figure', class_='wp-block-table')
    
    if figure_element:
        puissance = figure_element.find('tr', {'td': 'Puissance'}).find('td', {'td': 'Puissance'}).text.strip()
        autonomie = figure_element.find('tr', {'td': 'Autonomie (WLTP)'}).find('td', {'td': 'Autonomie (WLTP)'}).text.strip()
        prix_value = figure_element.find('tr', {'td': 'Prix'}).find('td', {'td': 'Prix'}).text.strip()
        bonus_value = figure_element.find('tr', {'td': 'Bonus'}).find('td', {'td': 'Bonus'}).text.strip()

        puissances.append(puissance)
        autonomies.append(autonomie)
        prix.append(prix_value)
        bonus.append(bonus_value)
    else:
        # If no <figure> element is found, append None for missing data
        puissances.append(None)
        autonomies.append(None)
        prix.append(None)
        bonus.append(None)

df = pd.DataFrame({
    'Modele': modeles,
    'Puissance': puissances,
    'Autonomie': autonomies,
    'Prix': prix,
    'Bonus': bonus
})

print(df)


Prix d’une voiture électrique et bonus écologique


AttributeError: 'NoneType' object has no attribute 'find'